# vCAT - Functional Localization


### Create feature space masks
#### Requires binarized FS masks for Fusiform and Parahippocampal

In [ ]:
#faces/fusiform

import os
from nipype.pipeline.engine import Workflow, Node, MapNode
from nipype.interfaces.utility import IdentityInterface, Function, Merge 
from nipype.interfaces.io import DataGrabber, DataSink
from nipype.interfaces.freesurfer.model import Binarize
from nipype.interfaces.freesurfer.preprocess import ApplyVolTransform
from nipype.interfaces.fsl.utils import ImageMaths

subjects_dir = '/home/data/madlab/Mattfeld_vCAT/derivatives/freesurfer'
work_dir = '/scratch/madlab/crash/mandy/vcat/roi_masks'
sink_dir = '/home/data/madlab/Mattfeld_vCAT/derivatives/feature_space'

subs = ['sub-005', 'sub-006', 'sub-007', 'sub-008', 'sub-010', 
        'sub-012', 'sub-013', 'sub-014', 'sub-015', 'sub-016', 
        'sub-018', 'sub-019', 'sub-020', 'sub-021', 'sub-022', 
        'sub-023', 'sub-024', 'sub-025', 'sub-026', 'sub-027', 
        'sub-028', 'sub-029', 'sub-030', 'sub-031', 'sub-032']
subs1 = ['sub-005']
wf = Workflow(name = 'wf')
wf.base_dir = work_dir

#iterate through subjects
subj_iterable = Node(IdentityInterface(fields = ['subject_id'], mandatory_inputs = True),
                     name = 'subj_iterable')
subj_iterable.iterables = ('subject_id', subs)


info = dict(ref_epi_file = [['subject_id']],
            bbreg_xfm = [['subject_id']],
            aparc_aseg = [['subject_id']],
            zstat = [['subject_id']])


#get necessary files
datasource = Node(DataGrabber(infields = ['subject_id'], outfields = list(info.keys())),
                  name = 'datasource')
datasource.inputs.base_directory = os.path.abspath('/home/data/madlab/Mattfeld_vCAT/')
datasource.inputs.field_template = dict(ref_epi_file = 'derivatives/preproc_loc/%s/ref/*.nii.gz',
                                        bbreg_xfm = 'derivatives/preproc_loc/%s/bbreg/_fs_register0/*.dat',
                                        aparc_aseg = 'derivatives/freesurfer/%s/mri/aparc+aseg.mgz', 
                                        zstat = 'derivatives/MVPA-Loc/lvl2/%s/fixedfx/zstat_faces_minus_maths.nii.gz')
datasource.inputs.template = '*'
datasource.inputs.template_args = info
datasource.inputs.sort_filelist = True
wf.connect(subj_iterable, 'subject_id', datasource, 'subject_id')


#binarize fusiform masks using FS labels (ANAT SPACE)
anat_mask = Node(Binarize(), 
                 name = 'anat_mask')
anat_mask.inputs.match = [1007, 2007] #fusiform                         
anat_mask.inputs.binary_file = 'bi-fusiform.nii.gz'
wf.connect(datasource, 'aparc_aseg', anat_mask, 'in_file')


#transform anatomical FS fusiform masks (EPI SPACE)
anatmask_xfm = Node(ApplyVolTransform(inverse = True, interp = 'nearest'),
                    name = 'anatmask_xfm')
anatmask_xfm.inputs.subjects_dir = subjects_dir
wf.connect(datasource, 'ref_epi_file', anatmask_xfm, 'source_file')
wf.connect(datasource, 'bbreg_xfm', anatmask_xfm, 'reg_file')
wf.connect(anat_mask, 'binary_file', anatmask_xfm, 'target_file')


#binarize face zstats based on thresholds (EPI SPACE)
zstat_mask = Node(Binarize(),
                  name = 'zstat_mask')
zstat_mask.inputs.subjects_dir = subjects_dir
zstat_mask.inputs.min = 3.2775
wf.connect(datasource, 'zstat', zstat_mask, 'in_file')


#binarize face zstats based on regional mask (EPI SPACE)
feature_mask = Node(ImageMaths(),
                    name = 'feature_mask')
feature_mask.inputs.op_string = '-mul'
wf.connect(zstat_mask, 'binary_file', feature_mask, 'in_file')
wf.connect(anatmask_xfm, 'transformed_file', feature_mask, 'in_file2')


#datasink node to save output
sink = Node(DataSink(), 
            name = "sink")
sink.inputs.base_directory = sink_dir
wf.connect(feature_mask, 'out_file', sink, 'features')
wf.run(plugin='SLURM', plugin_args={'sbatch_args':('-p investor --qos pq_madlab --account iacc_madlab'),'overwrite':True})

In [ ]:
#scenes/parahippocampal

import os
from nipype.pipeline.engine import Workflow, Node, MapNode
from nipype.interfaces.utility import IdentityInterface, Function, Merge 
from nipype.interfaces.io import DataGrabber, DataSink
from nipype.interfaces.freesurfer.model import Binarize
from nipype.interfaces.freesurfer.preprocess import ApplyVolTransform
from nipype.interfaces.fsl.utils import ImageMaths

subjects_dir = '/home/data/madlab/Mattfeld_vCAT/derivatives/freesurfer'
work_dir = '/scratch/madlab/crash/mandy/vcat/roi_masks'
sink_dir = '/home/data/madlab/Mattfeld_vCAT/derivatives/feature_space'

subs = ['sub-005', 'sub-006', 'sub-007', 'sub-008', 'sub-010', 
        'sub-012', 'sub-013', 'sub-014', 'sub-015', 'sub-016', 
        'sub-018', 'sub-019', 'sub-020', 'sub-021', 'sub-022', 
        'sub-023', 'sub-024', 'sub-025', 'sub-026', 'sub-027', 
        'sub-028', 'sub-029', 'sub-030', 'sub-031', 'sub-032']
subs1 = ['sub-005']
wf = Workflow(name = 'wf')
wf.base_dir = work_dir

#iterate through subjects
subj_iterable = Node(IdentityInterface(fields = ['subject_id'], mandatory_inputs = True),
                     name = 'subj_iterable')
subj_iterable.iterables = ('subject_id', subs)


#get necessary files
info = dict(ref_epi_file = [['subject_id']],
            bbreg_xfm = [['subject_id']],
            aparc_aseg = [['subject_id']],
            zstat = [['subject_id']])
datasource = Node(DataGrabber(infields = ['subject_id'], outfields = list(info.keys())),
                  name = 'datasource')
datasource.inputs.base_directory = os.path.abspath('/home/data/madlab/Mattfeld_vCAT/')
datasource.inputs.field_template = dict(ref_epi_file = 'derivatives/preproc_loc/%s/ref/*.nii.gz',
                                        bbreg_xfm = 'derivatives/preproc_loc/%s/bbreg/_fs_register0/*.dat',
                                        aparc_aseg = 'derivatives/freesurfer/%s/mri/aparc+aseg.mgz', 
                                        zstat = 'derivatives/MVPA-Loc/lvl2/%s/fixedfx/zstat_scenes_minus_maths.nii.gz')
datasource.inputs.template = '*'
datasource.inputs.template_args = info
datasource.inputs.sort_filelist = True
wf.connect(subj_iterable, 'subject_id', datasource, 'subject_id')


#binarize parahippocampal masks using FS labels (ANAT SPACE)
anat_mask = Node(Binarize(), 
                 name = 'anat_mask')
anat_mask.inputs.match = [1016, 2016] #fusiform                         
anat_mask.inputs.binary_file = 'bi-parahippocampal.nii.gz'
wf.connect(datasource, 'aparc_aseg', anat_mask, 'in_file')


#transform anatomical FS parahippocampal masks (EPI SPACE)
anatmask_xfm = Node(ApplyVolTransform(inverse = True, interp = 'nearest'),
                    name = 'anatmask_xfm')
anatmask_xfm.inputs.subjects_dir = subjects_dir
wf.connect(datasource, 'ref_epi_file', anatmask_xfm, 'source_file')
wf.connect(datasource, 'bbreg_xfm', anatmask_xfm, 'reg_file')
wf.connect(anat_mask, 'binary_file', anatmask_xfm, 'target_file')


#binarize scene zstats based on thresholds (EPI SPACE)
zstat_mask = Node(Binarize(),
                  name = 'zstat_mask')
zstat_mask.inputs.subjects_dir = subjects_dir
zstat_mask.inputs.min = 3.2775
wf.connect(datasource, 'zstat', zstat_mask, 'in_file')


#binarize scene zstats based on regional mask (EPI SPACE)
feature_mask = Node(ImageMaths(),
                    name = 'feature_mask')
feature_mask.inputs.op_string = '-mul'
wf.connect(zstat_mask, 'binary_file', feature_mask, 'in_file')
wf.connect(anatmask_xfm, 'transformed_file', feature_mask, 'in_file2')


#datasink node to save output
sink = Node(DataSink(), 
            name = "sink")
sink.inputs.base_directory = sink_dir
wf.connect(feature_mask, 'out_file', sink, 'features')
wf.run(plugin='SLURM', plugin_args={'sbatch_args':('-p investor --qos pq_madlab --account iacc_madlab'),'overwrite':True})

#### Move shit

In [ ]:
import os
from os.path import join, basename
from glob import glob
import shutil 

sids = ['sub-006', 'sub-007', 'sub-008', 'sub-010', 
        'sub-012', 'sub-013', 'sub-014', 'sub-015', 'sub-016', 
        'sub-018', 'sub-019', 'sub-020', 'sub-021', 'sub-022', 
        'sub-023', 'sub-024', 'sub-025', 'sub-026', 'sub-027', 
        'sub-028', 'sub-029', 'sub-030', 'sub-031', 'sub-032']
sids1 = ['sub-005']
for sid in sids:
    mask_dir = '/home/data/madlab/Mattfeld_vCAT/derivatives/feature_space/'
    
    if not os.path.exists(join(mask_dir, '{0}/'.format(sid))):
        os.makedirs(join(mask_dir, '{0}/'.format(sid)))

    shutil.move(join(mask_dir, 'features/_subject_id_{0}/zstat_scenes_minus_maths_thresh_maths.nii.gz'.format(sid)),
                join(mask_dir, '{0}/feature_scenes.nii.gz'.format(sid)))
               